<a href="https://colab.research.google.com/github/ethantrang/zalo_ai_challenge/blob/main/zalo_ai_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch]
!pip install datasets
!pip install pandas
!pip install torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
import pandas as pd
import torch
import json

In [ ]:
# Test inference
checkpoint = "bigscience/bloomz-560m"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

inputs = tokenizer.encode("Translate to English: Je t’aime.", return_tensors="pt")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

(…)-560m/resolve/main/tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

(…)60m/resolve/main/special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

(…)nce/bloomz-560m/resolve/main/config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translate to English: Je t’aime. I love you.</s>


In [17]:
# Loading bloomz model and tokenizer
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloomz-560m")
model = BloomForCausalLM.from_pretrained("bigscience/bloomz-560m").to("cuda")


In [ ]:
# Loading dataset prompts.json built using de portuguese legalQA dataset
dataset = load_dataset("json", data_files="/content/data/prompts.json")

In [18]:
# prepare the data for training
def prepare_train_data(data):
    # # prompt + completion
    # text_input = data['text']
    # # tokenize the input (prompt + completion) text
    # tokenized_input = tokenizer(text_input, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
    # # generative models: labels are the same as the input
    # tokenized_input['labels'] = tokenized_input['input_ids']
    # return tokenized_input

    # prompt + completion
    text_input = data['text']
    # tokenize the input (prompt + completion) text
    tokenized_input = tokenizer(text_input, return_tensors='pt', padding='max_length', max_length=128, truncation=True)

    # tokenize the label separately
    label_input = tokenizer(data['text'], return_tensors='pt', padding='max_length', max_length=128, truncation=True)

    # generative models: labels are the same as the input
    tokenized_input['labels'] = label_input['input_ids']

    return tokenized_input

train_dataset = dataset['train'].map(prepare_train_data,
                                     batched=True,
                                     remove_columns=["text"])

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [19]:
# setting arguments to be used during training
training_arguments = TrainingArguments(
    'first-model',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
    optim="adafactor",
    gradient_accumulation_steps=4,
    gradient_checkpointing=True
)

In [20]:
trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset
)

trainer.train()
trainer.save_model()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


In [22]:
# Testing new pretrained model
import torch
from transformers import pipeline
from transformers import BloomTokenizerFast, BloomForCausalLM

# Loading the fine-tuned model: LegalQA-bloom-560m
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloomz-560m")
model = BloomForCausalLM.from_pretrained("/content/first-model",
                                          low_cpu_mem_usage=True).to("cpu")


In [23]:
prompt = "Given the question:\nMột cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng. Hỏi nếu bán hết hàng thì cửa hàng thu được bao nhiêu tiền?\nA. 4 500 000 đồng\nB. 45 000 000 đồng\nC. 50 000 000 đồng\nD. 450 000 000 đồng\nThe answer is:"
generator = pipeline('text-generation',
                      model=model,
                      tokenizer=tokenizer,
                      do_sample=False)
result = generator(prompt, max_length=128)
print(result)

[{'generated_text': 'Given the question:\nMột cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng. Hỏi nếu bán hết hàng thì cửa hàng thu được bao nhiêu tiền?\nA. 4 500 000 đồng\nB. 45 000 000 đồng\nC. 50 000 000 đồng\nD. 450 000 000 đồng\nThe answer is: nan Vì vậy đáp án là: C. 50 000 000 đồng\n D. 450 000 000 đồng\n Đáp án là: C. 50 000 000 đồng\n Đáp án là: B. 45 000 000 đồng\n Đáp án là: C. 50 000 000 đồng\n Đáp án là: B. 45'}]
